In [0]:
from chispa.dataframe_comparer import assert_df_equality
from pyspark.sql import functions as F


from src.bronze_ingest import ingest_to_bronze


def test_ingest_to_bronze_adds_partition_col(spark, tmp_path):
   
    data = [
        ("o1", "2021-01-15 10:00:00"),
        ("o2", "2021-02-20 12:00:00"),
    ]
    df = spark.createDataFrame(data, ["order_id", "order_purchase_timestamp"]) \
              .withColumn("order_purchase_timestamp", F.to_timestamp("order_purchase_timestamp"))

        cfg = {"input_path": "", "output_path": str(tmp_path), "partition_col": "order_purchase_timestamp"}

   
    result = ingest_to_bronze(df, cfg)   

    expected = spark.createDataFrame([
        ("o1", "2021-01-15 10:00:00", "2021-01"),
        ("o2", "2021-02-20 12:00:00", "2021-02"),
    ], ["order_id", "order_purchase_timestamp", "order_month"]) \
    .withColumn("order_purchase_timestamp", F.to_timestamp("order_purchase_timestamp"))

    assert_df_equality(result, expected, ignore_row_order=True)
